# Machine learning avec des catégories et du texte

Le jeu [Titanic](http://biostat.mc.vanderbilt.edu/wiki/Main/DataSets) contient la liste des passages du Titanic, quelques informations comme le billet, la classe, et le fait qu'ils aient survécu. Il est possible d'utiliser le machine learning pour étudier la probabilité de survie ou plutôt de comprendre un peu mieux qui a eu la chance de survivre. S'il est possible de prédire, alors il existe une sorte de règle.

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [2]:
%matplotlib inline

## Les données

On peut les récupérer manuellement ou utiliser la fonction *load_titanic_dataset*. On pourra prendre l'un des deux jeux suivants.

In [3]:
from papierstat.datasets import load_titanic_dataset
data1 = load_titanic_dataset(subset="A")
data1.head(n=2)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"


In [4]:
data2 = load_titanic_dataset(subset="B")
data2.head(n=2)

,row.names,pclass,survived,name,age,embarked,home.dest,room,ticket,boat,sex
0,1,1st,1,"Allen, Miss Elisabeth Walton",29.0,Southampton,"St Louis, MO",B-5,24160 L221,2,female
1,2,1st,0,"Allison, Miss Helen Loraine",2.0,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female


## Premier modèle de prédiction

On veut savoir si la survie de chaque personne était plutôt aléatoire où certaines personnes ont été privilégiées. Plutôt que de se lancer dans une étude de statistique descriptive, on cale un modèle de prédiction. Si celui-ci fonctionne, cela signifie qu'il existe un lien entre la survie et certaines des informations connues sur chaque passager. On considère les variables ``age, sex, pclass``.

In [5]:
df = data1

In [6]:
from sklearn.model_selection import train_test_split

X = df[["age", "sex", "pclass"]]
y = df['survived']
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [7]:
set(df.sex), set(df.pclass)

({'female', 'male'}, {1, 2, 3})

Il faut d'abord transformer les variables catégorielles en variables numériques. C'est le rôle d'un [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) : chaque catégorie devient une variable binaire.

In [8]:
from sklearn.preprocessing import OneHotEncoder
one = OneHotEncoder()
one.fit(X_train[['sex', 'pclass']])

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [9]:
one.transform(X_train[['sex', 'pclass']])

<981x5 sparse matrix of type '<class 'numpy.float64'>'
	with 1962 stored elements in Compressed Sparse Row format>

Le résultat est une matrice sparse ou creuse qui ne contient que les valeurs non nulles. Pour voir la matrice, il faut la rendre dense.

In [10]:
one.transform(X_train[['sex', 'pclass']]).todense()

matrix([[1., 0., 0., 0., 1.],
        [0., 1., 0., 0., 1.],
        [1., 0., 1., 0., 0.],
        ...,
        [1., 0., 0., 0., 1.],
        [1., 0., 1., 0., 0.],
        [0., 1., 0., 0., 1.]])

Cinq colonnes correspondant aux cinq catégories dont les noms sont les suivants :

In [11]:
names = one.get_feature_names()
names

array(['x0_female', 'x0_male', 'x1_1', 'x1_2', 'x1_3'], dtype=object)

In [12]:
import numpy
cats = one.transform(X_train[['sex', 'pclass']]).todense()
age = X_train[['age']]
feat = numpy.hstack([age, cats])
feat[:10]

matrix([[ 1.,  1.,  0.,  0.,  0.,  1.],
        [37.,  0.,  1.,  0.,  0.,  1.],
        [29.,  1.,  0.,  1.,  0.,  0.],
        [27.,  0.,  1.,  0.,  1.,  0.],
        [21.,  1.,  0.,  0.,  0.,  1.],
        [57.,  1.,  0.,  0.,  1.,  0.],
        [nan,  0.,  1.,  0.,  0.,  1.],
        [45.,  1.,  0.,  1.,  0.,  0.],
        [32.,  0.,  1.,  0.,  0.,  1.],
        [nan,  0.,  1.,  0.,  0.,  1.]])

La colonne `age` contient des valeurs manquantes. On les remplace par la moyenne.

In [13]:
df[['age']].shape, df[['age']].dropna().shape

((1309, 1), (1046, 1))

In [14]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer()
new_age = imp.fit_transform(X_train[['age']])
feat = numpy.hstack([new_age, cats])
feat[:10]

matrix([[ 1.        ,  1.        ,  0.        ,  0.        ,  0.        ,
          1.        ],
        [37.        ,  0.        ,  1.        ,  0.        ,  0.        ,
          1.        ],
        [29.        ,  1.        ,  0.        ,  1.        ,  0.        ,
          0.        ],
        [27.        ,  0.        ,  1.        ,  0.        ,  1.        ,
          0.        ],
        [21.        ,  1.        ,  0.        ,  0.        ,  0.        ,
          1.        ],
        [57.        ,  1.        ,  0.        ,  0.        ,  1.        ,
          0.        ],
        [29.11533587,  0.        ,  1.        ,  0.        ,  0.        ,
          1.        ],
        [45.        ,  1.        ,  0.        ,  1.        ,  0.        ,
          0.        ],
        [32.        ,  0.        ,  1.        ,  0.        ,  0.        ,
          1.        ],
        [29.11533587,  0.        ,  1.        ,  0.        ,  0.        ,
          1.        ]])

On peut maintenant caler une forêt aléatoire.

In [15]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf.fit(feat, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [16]:
cats_test = one.transform(X_test[['sex', 'pclass']])
age_test = imp.transform(X_test[['age']])
feat_test = numpy.hstack([age_test, cats_test.todense()])

On regarde les prédictions sur la base de test et ça a l'air de marcher.

In [17]:
from sklearn.metrics import confusion_matrix

pred = rf.predict(feat_test)
confusion_matrix(y_test, pred)

array([[167,  38],
       [ 30,  93]], dtype=int64)

La survie n'est pas donc aléatoire. On met tout cela dans un pipeline car c'est toujours plus simple à lire.

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

pipe = Pipeline([
    ('cats', ColumnTransformer([
        ('one', OneHotEncoder(), ['sex', 'pclass']),
        ('imp', SimpleImputer(), ['age'])
    ]))
])
pipe.fit(X_train)
pipe.transform(X_test)

array([[ 0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
        26.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
        33.        ],
       [ 0.        ,  1.        ,  1.        ,  0.        ,  0.        ,
        29.11533587],
       ...,
       [ 0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
        32.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
        32.        ],
       [ 0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         2.        ]])

On modifie le pipeline pour ajouter la forêt aléatoire.

In [19]:
pipe = Pipeline([
    ('cats', ColumnTransformer([
        ('one', OneHotEncoder(), ['sex', 'pclass']),
        ('imp', SimpleImputer(), ['age'])
    ])),
    ('rf', RandomForestClassifier(n_estimators=100))
])

pipe.fit(X_train, y_train)
confusion_matrix(y_test, pipe.predict(X_test))

array([[181,  25],
       [ 43,  79]], dtype=int64)

C'est plus clair. Quelques explications sur les variables, mais il faut vérifier en les retirant une à une pour vérifier leur importance dans l'histoire.

In [20]:
cols = ['age'] + list(names)
list(zip(cols, pipe.steps[-1][1].feature_importances_))

[('age', 0.22307728202566554),
 ('x0_female', 0.2178965553376631),
 ('x0_male', 0.043462997658273654),
 ('x1_1', 0.021388326525194745),
 ('x1_2', 0.08636043368153981),
 ('x1_3', 0.40781440477166325)]

## Utiliser d'autres variables

La variable `ticket` contient d'autres informations mais comme chaque ticket est unique, il est difficile de l'utiliser telle quelle. Il faut chercher des redondances d'une ligne à l'autre autrement c'est inexploitable. Et la redondance vient des mots que cette colonne contient. Il faut découper en mots :

* On fait l'inventaire de tous les mots uniques : ce sont des catégories.
* On crée une variable par mot : 1 si l'expression contient le mot, 0 sinon.

C'est une approche dite *bag of words* ou *sac de mots*.

In [21]:
[_ for _ in set(df.ticket) if ' ' in _][:10]

['A/5 3536',
 'PC 17482',
 'PC 17477',
 'A/S 2816',
 'PC 17609',
 'PC 17476',
 'SOTON/O.Q. 3101263',
 'C.A. 17248',
 'STON/O 2. 3101286',
 'SO/C 14885']

In [22]:
X = df[["age", "sex", "pclass", "ticket"]]
y = df['survived']
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

pipe = Pipeline([
    ('cats', ColumnTransformer([
        ('one', OneHotEncoder(), ['sex', 'pclass']),
        ('imp', SimpleImputer(), ['age']),
        ('bow', CountVectorizer(), ['ticket']),
    ])),
])

try:
    pipe.fit(X_train)
except ValueError as e:
    print(e)

all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 981 and the array at index 2 has size 1


Le modèle [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) est un peu problèmatique car il ne traite qu'une seule colonne. Il faut ruser pour l'utiliser dans un pipeline.

In [24]:
CountVectorizer().fit_transform(X_train['ticket'])

<981x738 sparse matrix of type '<class 'numpy.int64'>'
	with 1166 stored elements in Compressed Sparse Row format>